In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd

In [46]:
# Plank dataset class
class PlankDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.data.iloc[idx].values, dtype=torch.float32)
        return features

In [47]:
# Plank CNN model

class PlankCNN(nn.Module):
    def __init__(self):
        super(PlankCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 8, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(8, 16, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(16, 7)  # Corrected input size

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 16)  # Corrected flattening
        x = self.fc1(x)
        return x

In [48]:
# data split and loader
dataset = PlankDataset('/Users/defeee/Documents/GitHub/FormAI-ML/Computer_Vision/plank_features.csv')
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [49]:
model = PlankCNN()
optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()

In [50]:
num_epochs = 10000
for epoch in range(num_epochs):
    # Training
    model.train()
    train_epoch_loss = 0.0
    for i, data in enumerate(train_dataloader):
        inputs = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_epoch_loss += loss.item()

    # Validation
    model.eval()
    val_epoch_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(val_dataloader):
            inputs = data
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            val_epoch_loss += loss.item()

    # Print training and validation losses
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_epoch_loss / len(train_dataloader):.4f}, '
          f'Val Loss: {val_epoch_loss / len(val_dataloader):.4f}')

Epoch [1/10000], Train Loss: 13198.8828, Val Loss: 13610.7012
Epoch [2/10000], Train Loss: 12924.6689, Val Loss: 13333.2822
Epoch [3/10000], Train Loss: 12661.4102, Val Loss: 13066.5322
Epoch [4/10000], Train Loss: 12415.3623, Val Loss: 12815.6367
Epoch [5/10000], Train Loss: 12180.1143, Val Loss: 12571.1748
Epoch [6/10000], Train Loss: 11950.7373, Val Loss: 12330.8604
Epoch [7/10000], Train Loss: 11725.8008, Val Loss: 12095.0527
Epoch [8/10000], Train Loss: 11504.6445, Val Loss: 11863.8438
Epoch [9/10000], Train Loss: 11288.6406, Val Loss: 11640.3818
Epoch [10/10000], Train Loss: 11080.3594, Val Loss: 11420.7773
Epoch [11/10000], Train Loss: 10878.0264, Val Loss: 11206.6562
Epoch [12/10000], Train Loss: 10679.6572, Val Loss: 10995.3750
Epoch [13/10000], Train Loss: 10484.4736, Val Loss: 10786.7822
Epoch [14/10000], Train Loss: 10292.3252, Val Loss: 10580.7070
Epoch [15/10000], Train Loss: 10103.0352, Val Loss: 10376.9932
Epoch [16/10000], Train Loss: 9915.8291, Val Loss: 10175.4141
Ep

In [51]:
# Save model
torch.save(model.state_dict(), '/Users/defeee/Documents/GitHub/FormAI-ML/Models/Core/Plank/plank_cnn.pth')